In [100]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

# 1. POPULASI

In [101]:
#SAMPLE
# Specify the path to your .xlsx file
# file_path = "C://Users//LENOVO//Downloads//STATIN_DATA//sample_wildfire_weather_data.csv"

# # Read the CSV file
# df = pd.read_csv(file_path)


#POPULASI
file_path = "C://Users//LENOVO//Downloads//STATIN_DATA\wildfire_weather_data.xlsx"

# Read the Excel file
df = pd.read_excel(file_path, engine='openpyxl')

# Display the DataFrame
df.head()

,Region,Estimated_fire_area,Temperature_Mean,WindSpeed_Mean
0,NSW,8.68000,23.055527,4.841764
1,NSW,16.61125,22.425765,4.014080
2,NSW,5.52000,20.621405,4.477879
3,NSW,6.26400,19.632722,4.017578
4,NSW,5.40000,21.477315,3.884550


# Populasi Temperature

In [102]:
temp_df = df[['Region','Temperature_Mean']]
print(temp_df)

      Region  Temperature_Mean
0        NSW         23.055527
1        NSW         22.425765
2        NSW         20.621405
3        NSW         19.632722
4        NSW         21.477315
...      ...               ...
26401     WA         27.784354
26402     WA         27.351996
26403     WA         26.532585
26404     WA         26.231115
26405     WA         28.090392

[26406 rows x 2 columns]


In [103]:
total_value = temp_df.Temperature_Mean.sum()
jumlah_data = len(temp_df.Region)
print(f"Total populasi value temperature keseluruhan (T2.. untuk pengurang di SST dan SSK): {total_value} | N (untuk pengurang di SST dan SSK): {jumlah_data }")

Total populasi value temperature keseluruhan (T2.. untuk pengurang di SST dan SSK): 563224.643466062 | N (untuk pengurang di SST dan SSK): 26406


In [104]:
temp_pivot = temp_df.pivot(index=None, columns='Region', values='Temperature_Mean')
temp_pivot_reset  = temp_pivot.reset_index(drop=True)
temp_pivot_reset

Region,NSW,NT,QL,SA,TA,VI,WA
0,23.055527,NaN,NaN,NaN,NaN,NaN,NaN
1,22.425765,NaN,NaN,NaN,NaN,NaN,NaN
2,20.621405,NaN,NaN,NaN,NaN,NaN,NaN
3,19.632722,NaN,NaN,NaN,NaN,NaN,NaN
4,21.477315,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
26401,NaN,NaN,NaN,NaN,NaN,NaN,27.784354
26402,NaN,NaN,NaN,NaN,NaN,NaN,27.351996
26403,NaN,NaN,NaN,NaN,NaN,NaN,26.532585
26404,NaN,NaN,NaN,NaN,NaN,NaN,26.231115


In [105]:
column_sums = temp_pivot_reset.sum()

# Count non-NaN values for each column
non_nan_counts = temp_pivot_reset.count()
print("Sum of each column (T2..untuk SSK):")
print(column_sums)

print("\nCount of non-NaN values for each column (n untuk SSK):")
print(non_nan_counts)

Sum of each column (T2..untuk SSK):
Region
NSW     82690.270742
NT     126321.040737
QL     129660.952214
SA      43948.237472
TA      16889.975475
VI      33895.191487
WA     129818.975338
dtype: float64

Count of non-NaN values for each column (n untuk SSK):
Region
NSW    4621
NT     5052
QL     5531
SA     1989
TA     1404
VI     2175
WA     5625
dtype: int64


In [106]:
model = ols('Temperature_Mean ~ (Region)', data=temp_df).fit()
temp_anova_table = sm.stats.anova_lm(model, typ=1)
temp_anova_table

,df,sum_sq,mean_sq,F,PR(>F)
Region,6.0,359041.915432,59840.319239,1979.878018,0.0
Residual,26390.0,797617.838352,30.224245,NaN,NaN


In [107]:
from scipy.stats import f
alfa = 0.05
k = len(temp_df.Region.unique())
N = len(temp_df)
v1 = k-1
v2 = N-k
v_tot = v1+v2
SSK = temp_anova_table['sum_sq']['Region']
SSE = temp_anova_table['sum_sq']['Residual']
SST = SSK+SSE
MSK = SSK/v1
MSE = SSE/v2
F_hitung = MSK/MSE
F_table = f.ppf(1 - alfa, v1, v2)
sum_value = temp_df.Temperature_Mean.sum()

temp_data = {
    'SUMBER VARIANSI': ['Perlakuan', 'Error', 'TOTAL'],
    'SS': [SSK, SSE, SST],
    'v': [v1, v2, v_tot],
    'MS': [MSK, MSE, None],
    'Fhitung': [F_hitung, None, None]
}

# Create a DataFrame
temp_anova_final = pd.DataFrame(temp_data )

# Display the DataFrame
print(temp_anova_final)
print(f"Diketahui: ")
print(f"Total Sum / Sigma Xi = {sum_value}")
print(f"alfa: {alfa}")
print(f"F hitung = {MSK}/{MSE} = {F_hitung}")
print(f"F tabel = F({alfa};{v1};{v2}) = {F_table}")
if F_hitung > F_table:
    print("Maka h0 ditolak, artinya ada minimal 1 region yang berbeda nilainya")
elif F_hitung < F_table:
    print("Maka h0 diterima, artinya semua region sama nilainya")

  SUMBER VARIANSI            SS      v            MS      Fhitung
0       Perlakuan  3.590419e+05      6  59840.319239  1980.553232
1           Error  7.976178e+05  26399     30.213941          NaN
2           TOTAL  1.156660e+06  26405           NaN          NaN
Diketahui: 
Total Sum / Sigma Xi = 563224.643466062
alfa: 0.05
F hitung = 59840.31923871817/30.213941374759262 = 1980.5532319166011
F tabel = F(0.05;6;26399) = 2.098939395708575
Maka h0 ditolak, artinya ada minimal 1 region yang berbeda nilainya


# POPULASI WindSpeed


In [108]:
windspd_df = df[['Region','WindSpeed_Mean']]
print(windspd_df)

      Region  WindSpeed_Mean
0        NSW        4.841764
1        NSW        4.014080
2        NSW        4.477879
3        NSW        4.017578
4        NSW        3.884550
...      ...             ...
26401     WA        4.268756
26402     WA        4.365700
26403     WA        4.496722
26404     WA        4.211237
26405     WA        4.373947

[26406 rows x 2 columns]


In [109]:
total_value = windspd_df.WindSpeed_Mean.sum()
jumlah_data = len(windspd_df.WindSpeed_Mean)
print(f"Total populasi value temperature keseluruhan (T2.. untuk pengurang di SST dan SSK): {total_value} | N (untuk pengurang di SST dan SSK): {jumlah_data }")

Total populasi value temperature keseluruhan (T2.. untuk pengurang di SST dan SSK): 98692.96743262798 | N (untuk pengurang di SST dan SSK): 26406


In [110]:
windspd_pivot = windspd_df.pivot(index=None, columns='Region', values='WindSpeed_Mean')
windspd_pivot_reset  = windspd_pivot.reset_index(drop=True)
windspd_pivot_reset

Region,NSW,NT,QL,SA,TA,VI,WA
0,4.841764,NaN,NaN,NaN,NaN,NaN,NaN
1,4.014080,NaN,NaN,NaN,NaN,NaN,NaN
2,4.477879,NaN,NaN,NaN,NaN,NaN,NaN
3,4.017578,NaN,NaN,NaN,NaN,NaN,NaN
4,3.884550,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
26401,NaN,NaN,NaN,NaN,NaN,NaN,4.268756
26402,NaN,NaN,NaN,NaN,NaN,NaN,4.365700
26403,NaN,NaN,NaN,NaN,NaN,NaN,4.496722
26404,NaN,NaN,NaN,NaN,NaN,NaN,4.211237


In [111]:
column_sums = windspd_pivot_reset.sum()

# Count non-NaN values for each column
non_nan_counts = windspd_pivot_reset.count()
print("Sum of each column (T2..untuk SSK):")
print(column_sums)

print("\nCount of non-NaN values for each column (n untuk SSK):")
print(non_nan_counts)

Sum of each column (T2..untuk SSK):
Region
NSW    15254.574926
NT     19816.319804
QL     19291.974251
SA      8226.410201
TA      4894.513913
VI      7126.992661
WA     24082.181677
dtype: float64

Count of non-NaN values for each column (n untuk SSK):
Region
NSW    4620
NT     5049
QL     5529
SA     1990
TA     1403
VI     2176
WA     5623
dtype: int64


In [112]:
model = ols('WindSpeed_Mean ~ (Region)', data=windspd_df).fit()
windspd_anova_table = sm.stats.anova_lm(model, typ=1)
windspd_anova_table

,df,sum_sq,mean_sq,F,PR(>F)
Region,6.0,3931.072268,655.178711,902.448842,0.0
Residual,26383.0,19154.082911,0.726001,NaN,NaN


In [113]:
from scipy.stats import f
alfa = 0.05
k = len(temp_df.Region.unique())
N = len(temp_df)
v1 = k-1
v2 = N-k
v_tot = v1+v2
SSK = windspd_anova_table['sum_sq']['Region']
SSE = windspd_anova_table['sum_sq']['Residual']
SST = SSK+SSE
MSK = SSK/v1
MSE = SSE/v2
F_hitung = MSK/MSE
F_table = f.ppf(1 - alfa, v1, v2)

temp_data = {
    'SUMBER VARIANSI': ['Perlakuan', 'Error', 'TOTAL'],
    'SS': [SSK, SSE, SST],
    'v': [v1, v2, v_tot],
    'MS': [MSK, MSE, None],
    'Fhitung': [F_hitung, None, None]
}

# Create a DataFrame
temp_anova_final = pd.DataFrame(temp_data )

# Display the DataFrame
print(temp_anova_final)
print(f"Diketahui: ")
print(f"alfa: {alfa}")
print(f"F hitung = {MSK}/{MSE} = {F_hitung}")
print(f"F tabel = F({alfa};{v1};{v2}) = {F_table}")
if F_hitung > F_table:
    print("Maka h0 ditolak, artinya ada minimal 1 region yang berbeda nilainya")
elif F_hitung < F_table:
    print("Maka h0 diterima, artinya semua region sama nilainya")

  SUMBER VARIANSI            SS      v          MS     Fhitung
0       Perlakuan   3931.072268      6  655.178711  902.996133
1           Error  19154.082911  26399    0.725561         NaN
2           TOTAL  23085.155179  26405         NaN         NaN
Diketahui: 
alfa: 0.05
F hitung = 655.1787113779014/0.7255609269525384 = 902.9961331155848
F tabel = F(0.05;6;26399) = 2.098939395708575
Maka h0 ditolak, artinya ada minimal 1 region yang berbeda nilainya
